<a href="https://colab.research.google.com/github/Tabs2206/Meteor-Detection/blob/main/aircraft_model/Aircraft_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aircraft Classification Using Polygon Annotation


We employed the LabelMe application to annotate our dataset. To prepare the data for input into the YOLOv5 model, the JSON annotations were converted into text format as follows:

Using the minimum and maximum values of the x and y axes, a rectangular bounding box was created around each labeled image. This box was then represented in the text file with the format:

<class> <x-axis> <y-axis> <width> <height>

This format was used to facilitate the model's processing of the labeled data.

In [ ]:
import json
import os

def convert_json_to_yolo(json_file_path, output_folder, image_size=(860, 538)):
    """
    Convert JSON annotations to YOLO format text files.

    Args:
    - json_file_path: Path to the JSON file containing annotations.
    - output_folder: Path to the folder where YOLO format text files will be saved.
    - image_size: Tuple of (width, height) of the images, used for normalization.
    """
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Iterate over each image in the JSON data
    for image_name, image_data in data.items():
        regions = image_data.get('regions', {})
        yolo_label_file = os.path.join(output_folder, image_name.replace('.png', '.txt'))

        with open(yolo_label_file, 'w') as f_out:
            for region in regions.values():
                points_x = region['shape_attributes']['all_points_x']
                points_y = region['shape_attributes']['all_points_y']

                # Calculate bounding box from polygon points
                x_min, x_max = min(points_x), max(points_x)
                y_min, y_max = min(points_y), max(points_y)

                # Calculate center, width, and height
                x_center = (x_min + x_max) / 2.0 / image_size[0]
                y_center = (y_min + y_max) / 2.0 / image_size[1]
                width = (x_max - x_min) / image_size[0]
                height = (y_max - y_min) / image_size[1]

                # Assuming a single class (e.g., 'aircraft') with class_id 0
                f_out.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Paths to JSON files and output directories
train_json_path = '/content/drive/MyDrive/class_aircraft/labels/final100.json'
# val_json_path = '/content/drive/MyDrive/class_aircraft/labels/YOLOJSONvval.json'

train_labels_path = '/content/drive/MyDrive/class_aircraft/labels/train'
# val_labels_path = '/content/drive/MyDrive/class_aircraft/labels/val'

# Convert JSON files to YOLO format
convert_json_to_yolo(train_json_path, train_labels_path)
convert_json_to_yolo(val_json_path, val_labels_path)

# Verify conversion by listing some of the YOLO files
yolo_train_files = os.listdir(train_labels_path)
yolo_val_files = os.listdir(val_labels_path)

print("Sample YOLO Train Files:", yolo_train_files[:5])
print("Sample YOLO Val Files:", yolo_val_files[:5])


Sample YOLO Train Files: ['RAD_BEDOUR_20240604_0200_FRMAUB_SYS001.txt', 'RAD_BEDOUR_20240604_0205_FRMAUB_SYS001.txt', 'RAD_BEDOUR_20240604_0210_BEHUMA_SYS004.txt', 'RAD_BEDOUR_20240604_0210_FRMAUB_SYS001.txt', 'RAD_BEDOUR_20240604_0215_FRMAUB_SYS001.txt']
Sample YOLO Val Files: ['RAD_BEDOUR_20240611_0015_BEHUMA_SYS003.txt', 'RAD_BEDOUR_20240604_0235_BETINY_SYS001.txt', 'RAD_BEDOUR_20240611_0010_BEHUMA_SYS004.txt', 'RAD_BEDOUR_20240604_0240_BETINY_SYS001.txt', 'RAD_BEDOUR_20240604_0230_BETINY_SYS001.txt']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt   # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 33.6/201.2 GB disk)


Add the below code in the folder path "/content/yolov5/data"



```
train: /content/drive/MyDrive/class_aircraft/images/train # path to training images
val: /content/drive/MyDrive/class_aircraft/images/val   # path to validation images
test:     # path to test images (optional)


nc: 1  # number of classes
names: ['aircraft_curve']  # class names


```



In [3]:
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 200 --data /content/yolov5/data/coco128_aircraft.yaml.txt --weights yolov5s.pt --name aircraft_detection

2024-08-19 00:23:46.775660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 00:23:46.796269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 00:23:46.802695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/coco128_aircraft.yaml.txt, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, mult

In [5]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/aircraft_detection/weights/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/class_multi_1/images/train/RAD_BEDOUR_20240613_0045_BEOPHA_SYS001.png

detect: weights=['/content/yolov5/runs/train/aircraft_detection/weights/best.pt'], source=/content/drive/MyDrive/class_multi_1/images/train/RAD_BEDOUR_20240613_0045_BEOPHA_SYS001.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/class_multi_1/images/train/RAD_BEDOUR_20240613_0045_BEOPHA_SYS001.png: 416x640 7 aircraft_curves, 48.8ms
Speed: 0.5ms pre-process, 48.8ms inference, 521.0ms NMS per image at shape (1, 3, 640, 

# Saving our model data and output

#### we created a directory and copying our results and model to it.

In [6]:
!mkdir -p /content/drive/MyDrive/aircraftyolov5

In [7]:
save_path = '/content/drive/MyDrive/aircraftyolov5'

In [8]:
!cp /content/yolov5/runs/train/aircraft_detection/weights/best.pt {save_path}/best.pt
!cp /content/yolov5/runs/train/aircraft_detection/weights/last.pt {save_path}/last.pt

In [9]:
!cp -r /content/yolov5/runs/train/aircraft_detection {save_path}/traindata

In [10]:
!cp /content/yolov5/runs/detect/exp/RAD_BEDOUR_20240619_0210_BENEUF_SYS001.png {save_path}/output
!cp /content/yolov5/runs/detect/exp2/RAD_BEDOUR_20240613_0045_BEOPHA_SYS001.png {save_path}/output1